# Récupère 5 offres d'emplois sur hellowork à partir d'un mot clé (ici : Data Analyst)

In [ ]:
import time
import pandas as pd
import urllib.parse
import re  # Import nécessaire pour le nettoyage des espaces
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By

# --- 1. FONCTION DE NETTOYAGE (NOUVEAU) ---
def clean_text(text):
    """
    Nettoie le texte pour garantir qu'il tient sur une seule ligne.
    - Remplace les sauts de ligne (\n, \r) par des espaces.
    - Supprime les espaces multiples inutiles.
    """
    if not isinstance(text, str):
        return str(text) if text is not None else ""
    
    # Remplace les retours à la ligne par un espace simple
    text = text.replace('\n', ' ').replace('\r', ' ')
    
    # Remplace les suites d'espaces (ex: "  ") par un seul espace
    text = re.sub(r'\s+', ' ', text)
    
    return text.strip()

# --- 2. FONCTION POUR SÉPARER LE TEXTE ---
def extract_mission_profil(full_text):
    """
    Essaie de couper le texte en deux parties : Missions et Profil.
    Renvoie les textes nettoyés (sans sauts de ligne).
    """
    if not full_text:
        return "", ""

    text_lower = full_text.lower()
    
    profil_keywords = ["profil", "ce que nous recherchons", "votre profil", "compétences", "pré-requis", "attendus"]
    
    split_index = -1
    for keyword in profil_keywords:
        idx = text_lower.find(keyword)
        if idx != -1:
            # On évite de couper si le mot clé est au tout début (ex: titre)
            if idx > len(full_text) * 0.1: 
                split_index = idx
                break
    
    if split_index != -1:
        missions = full_text[:split_index]
        profil = full_text[split_index:]
    else:
        missions = full_text
        profil = "Non identifié spécifiquement (voir colonne Missions)"
        
    # IMPORTANT : On nettoie le texte avant de le renvoyer
    return clean_text(missions), clean_text(profil)

# --- 3. FONCTION PRINCIPALE ---
def get_jobs_detailed_hellowork(keyword, num_jobs=10):
    print(f"🚀 Démarrage de la recherche approfondie pour : {keyword}...")
    
    encoded_keyword = urllib.parse.quote_plus(keyword)
    url = f"https://www.hellowork.com/fr-fr/emploi/recherche.html?k={encoded_keyword}"
    
    options = Options()
    options.add_argument("--start-maximized")
    # User agent pour éviter d'être bloqué trop vite
    options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36")
    options.add_argument("--disable-blink-features=AutomationControlled")
    
    driver = webdriver.Edge(options=options)
    all_jobs_data = []
    
    try:
        # --- PHASE 1 : RÉCUPÉRATION DES LIENS ET DU LIEU ---
        print("1️⃣  PHASE 1 : Récupération des liens et des lieux...")
        driver.get(url)
        time.sleep(4)
        
        # Gestion des cookies
        try:
            buttons = driver.find_elements(By.TAG_NAME, "button")
            for btn in buttons:
                if "continuer sans" in btn.text.lower() or "refuser" in btn.text.lower():
                    btn.click()
                    break
            time.sleep(1)
        except: pass

        potential_jobs = driver.find_elements(By.CSS_SELECTOR, "ul > li")
        job_links = []
        
        for card in potential_jobs:
            if len(job_links) >= num_jobs:
                break
            try:
                if card.find_elements(By.TAG_NAME, "h3"):
                    link_elem = card.find_element(By.TAG_NAME, "a")
                    link = link_elem.get_attribute("href")
                    
                    # Titre
                    raw_title = card.find_element(By.TAG_NAME, "h3").text.split('\n')[0]
                    
                    # Extraction Entreprise et Lieu via le texte brut de la carte
                    txt_lines = card.text.split('\n')
                    
                    # L'entreprise est souvent en 2ème ligne
                    raw_company = txt_lines[1] if len(txt_lines) > 1 else "N/A"
                    if raw_company == raw_title and len(txt_lines) > 2: 
                         raw_company = txt_lines[2]

                    # Le lieu est souvent en 3ème ligne
                    raw_location = "N/A"
                    if len(txt_lines) > 2:
                        potential_loc = txt_lines[2]
                        if len(potential_loc) < 50: 
                            raw_location = potential_loc
                    
                    # On stocke les versions nettoyées
                    job_links.append({
                        "Poste": clean_text(raw_title),
                        "Entreprise": clean_text(raw_company),
                        "Lieu": clean_text(raw_location),
                        "Lien": link
                    })
            except:
                continue
                
        print(f"✅ {len(job_links)} liens trouvés. Passage à l'extraction détaillée.")
        
        # --- PHASE 2 : VISITE DE CHAQUE PAGE ---
        print("2️⃣  PHASE 2 : Analyse détaillée des annonces (Missions/Profil)...")
        
        for index, job in enumerate(job_links):
            print(f"   ⏳ Traitement {index + 1}/{len(job_links)} : {job['Poste']}...")
            
            try:
                driver.get(job['Lien'])
                time.sleep(3) 
                
                full_desc = ""
                try:
                    sections = driver.find_elements(By.TAG_NAME, "section")
                    # On prend la section avec le plus de texte, souvent la description
                    longest_section = max(sections, key=lambda x: len(x.text) if x.text else 0)
                    full_desc = longest_section.text
                except:
                    full_desc = driver.find_element(By.TAG_NAME, "body").text

                # Extraction et nettoyage via la fonction modifiée
                missions, profil = extract_mission_profil(full_desc)
                
                all_jobs_data.append({
                    "Poste": job['Poste'],
                    "Entreprise": job['Entreprise'],
                    "Lieu": job['Lieu'],
                    "Missions": missions,  # Déjà nettoyé par extract_mission_profil
                    "Profil_Recherche": profil, # Déjà nettoyé par extract_mission_profil
                    "Lien": job['Lien']
                })
                
            except Exception as e:
                print(f"   ❌ Erreur sur ce lien : {e}")
                job_fallback = job.copy()
                job_fallback["Missions"] = "Erreur extraction"
                job_fallback["Profil_Recherche"] = "Erreur extraction"
                all_jobs_data.append(job_fallback)

    except Exception as e:
        print(f"❌ Erreur critique : {e}")
        
    finally:
        driver.quit()
        print("✅ Terminé.")

    return pd.DataFrame(all_jobs_data)

# --- EXECUTION ET SAUVEGARDE ---
if __name__ == "__main__":
    # Lancer le scraping
    df_details = get_jobs_detailed_hellowork("Data Analyst", 5)

    # Affichage de contrôle
    pd.set_option('display.max_colwidth', 50)
    print("\nAperçu des données :")
    print(df_details[["Poste", "Lieu", "Missions"]].head())

    # Export CSV "PROPRE"
    nom_fichier = "hellowork_detailed_jobs_clean2.csv"
    
    # encoding='utf-8-sig' est important pour qu'Excel lise bien les accents
    # index=False évite d'avoir une colonne de numéros de ligne (0, 1, 2...)
    df_details.to_csv(nom_fichier, index=False, encoding='utf-8-sig', sep=',')
    
    print(f"\n📁 Fichier CSV propre généré : {nom_fichier}")

# Reformule les annonce grace à Qwen 2.5 1.5B 

In [ ]:
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

# 1. Configuration du modèle
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

print(f"Chargement du modèle {model_name}...")
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)

# 2. Chargement des données
try:
    df = pd.read_csv('hellowork_detailed_jobs_clean2.csv')
    print(f"CSV chargé : {len(df)} offres trouvées.")
except FileNotFoundError:
    print("Erreur : Fichier CSV non trouvé.")
    exit()

# Liste pour stocker les résultats au fur et à mesure
resumes_stockes = []

# 3. Boucle avec Streaming
print("\n=== DÉBUT DE LA GÉNÉRATION EN DIRECT ===\n")

# On utilise un iterrows() pour traiter ligne par ligne
for index, row in df.iterrows():
    
    # Affichage visuel pour séparer les offres dans le terminal
    print(f"\n\n--- Traitement de l'offre {index + 1}/{len(df)} : {row['Poste']} chez {row['Entreprise']} ---")
    print("RÉPONSE EN DIRECT : ", end="", flush=True)

    prompt = f"""Tu es un expert en recrutement. Analyse l'offre d'emploi ci-dessous et génère un résumé structuré.
    
    Données de l'offre :
    - Poste : {row['Poste']}
    - Entreprise : {row['Entreprise']}
    - Lieu : {row['Lieu']}
    - Missions : {row['Missions']}
    - Profil Recherché : {row['Profil_Recherche']}




    Génère la réponse uniquement sous ce format strict :
    RESUME_MATCHING:
    - Type de profil recherché
    - compétences clés: [Liste]
    - Soft_Skills: [Liste]
    - Seniority: [Niveau]
    - Core_Mission: [Phrase résumée]
    """

    messages = [
        {"role": "system", "content": "Tu es un assistant utile."},
        {"role": "user", "content": prompt}
    ]

    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

    # C'est ici que la magie opère : Le Streamer
    # skip_prompt=True permet de ne pas ré-afficher le prompt, juste la réponse de l'IA
    streamer = TextStreamer(tokenizer, skip_prompt=True)

    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=500,
        temperature=0.1,
        do_sample=True,
        streamer=streamer # <--- On active l'affichage en direct ici
    )
    
    # On doit quand même décoder le résultat pour le sauvegarder dans le CSV
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    response_text = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    # Nettoyage et stockage dans la liste
    clean_response = response_text.replace('RESUME_MATCHING:', '').strip()

    # --- MODIFICATION ICI ---
    # Remplacement des sauts de ligne (\n) par un séparateur unique (" | ")
    # Cela permet d'avoir tout le résumé sur une seule ligne physique dans le CSV
    clean_response_oneline = clean_response.replace('\n', ' | ').replace('\r', '')

    resumes_stockes.append(clean_response_oneline)

# 4. Sauvegarde finale
df['Resume_IA'] = resumes_stockes
df.to_csv('offres_resumees_qwen_live.csv', index=False)
print("\n\n=== TERMINÉ : Fichier sauvegardé ===")

# Transforme un CV pdf en .txt

In [ ]:
import pdfplumber
import re
import os
from typing import Optional

# --- CONFIGURATION ---
# Remplace par le chemin exact de ton fichier
CHEMIN_PDF = "cv Quénel.pdf" 
# ---------------------

def nettoyer_texte_avance(texte: str) -> str:
    """
    Nettoie les artefacts courants d'extraction PDF :
    - Recolle les lettres espacées (ex: "L a n g u e s" -> "Langues")
    - Supprime les sauts de ligne excessifs
    - Normalise les espaces
    """
    if not texte:
        return ""

    # 1. Correction du "Kerning" (Lettres espacées : L a n g u e s ou 0 6 1 1...)
    # Cette regex cherche une séquence de lettres/chiffres isolés séparés par un espace
    def replacer_lettres_isolees(match):
        return match.group(0).replace(" ", "")
    
    # Motif : (Caractère + Espace) répété au moins 3 fois, suivi d'un caractère
    texte = re.sub(r'(?:\b[A-Za-z0-9À-ÿ]\s){3,}[A-Za-z0-9À-ÿ]\b', replacer_lettres_isolees, texte)

    # 2. Remplacer les sauts de ligne multiples par un seul saut
    texte = re.sub(r'\n\s*\n', '\n\n', texte)

    # 3. Remplacer les espaces multiples (horizontaux) par un seul espace
    texte = re.sub(r'[ \t]+', ' ', texte)

    return texte.strip()

def extraire_cv_pro(chemin_fichier: str) -> Optional[str]:
    """
    Extrait le texte d'un CV en respectant au mieux la mise en page (colonnes).
    Utilise pdfplumber pour une précision professionnelle.
    """
    # Vérification de l'existence du fichier
    if not os.path.exists(chemin_fichier):
        print(f"❌ Erreur : Le fichier '{chemin_fichier}' est introuvable.")
        return None

    texte_global = ""
    
    try:
        print(f"🔄 Traitement de '{chemin_fichier}' avec pdfplumber...")
        
        with pdfplumber.open(chemin_fichier) as pdf:
            if not pdf.pages:
                print("⚠️ Le PDF semble vide.")
                return None

            for i, page in enumerate(pdf.pages):
                # extract_text() de pdfplumber gère mieux les colonnes que pypdf
                texte_page = page.extract_text(x_tolerance=2, y_tolerance=2)
                
                if texte_page:
                    texte_global += f"\n--- PAGE {i+1} ---\n"
                    texte_global += nettoyer_texte_avance(texte_page)
                else:
                    print(f"⚠️ Page {i+1} : Aucun texte sélectionnable (image scan ?).")

        print("✅ Extraction terminée avec succès.")
        return texte_global

    except Exception as e:
        print(f"❌ Une erreur critique est survenue : {e}")
        return None

# --- EXÉCUTION ---

# Appel de la fonction
contenu_cv = extraire_cv_pro(CHEMIN_PDF)

# Affichage du résultat si l'extraction a fonctionné
if contenu_cv:
    print("\n" + "="*40)
    print("      APERÇU DU CONTENU EXTRAIT")
    print("="*40 + "\n")
    print(contenu_cv)
    
    # Optionnel : Sauvegarde dans un fichier texte pour vérification
    with open("resultat_cv_quenel.txt", "w", encoding="utf-8") as f:
        f.write(contenu_cv)
    print("\n💾 Le résultat a également été sauvegardé dans 'resultat_cv_celyann.txt'.")

# Reformule le cv avec Qwen 2.5 1.5B

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
import os

# Configuration du modèle
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct"

def get_device():
    """Détermine le meilleur périphérique disponible (CUDA, MPS ou CPU)."""
    if torch.cuda.is_available(): return "cuda"
    if torch.backends.mps.is_available(): return "mps"
    return "cpu"

def synthesize_cv_for_matching(file_path):
    # 1. Lecture du fichier
    if not os.path.exists(file_path):
        print(f"Erreur : Le fichier '{file_path}' est introuvable.")
        return None
    
    with open(file_path, 'r', encoding='utf-8') as f:
        cv_content = f.read()

    print("Chargement du modèle en cours... (cela peut prendre quelques secondes)")
    
    # 2. Chargement du modèle et du tokenizer
    device = get_device()
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
    model = AutoModelForCausalLM.from_pretrained(MODEL_ID, device_map=device)

    # Streamer pour l'affichage en direct
    streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

    # 3. LE PROMPT "SYNTHÈSE & MATCHING"
    system_prompt = """Tu es un assistant de synthèse RH.
    Ta fonction est de transformer les informations professionnelles d'un CV en une fiche de compétences standardisée.
    Tu dois reformuler le contenu pour qu'il soit totalement neutre et générique.
    Remplace systématiquement l'identité du candidat par le terme : "Candidat".
    Concentre-toi uniquement sur les savoir-faire, les diplômes et l'expérience métier."""

    # ON INVERSE : Le CV est maintenant tout en haut
    user_prompt = f"""
    DOCUMENT À ANALYSER :
    ---
    {cv_content}
    ---

    INSTRUCTIONS :
    A partir du texte ci-dessus, extrais et classe les informations professionnelles.
    Remplis STRICTEMENT le modèle ci-dessous.
    Ne répète pas le texte original. Arrête-toi après la section 5.

    MODELE À REMPLIR :

    ### 1. Synthèse du Profil
    **Intitulé du poste** : [Indiquer le métier principal ici]
    **Resumé** : Candidat expérimenté dans le domaine de [Indiquer le secteur].

    ### 2. Compétences Techniques (Hard Skills)
    [Lister les logiciels, outils et techniques métier]

    ### 3. Qualités Professionnelles (Soft Skills)
    [Lister les qualités humaines et relationnelles]

    ### 4. Analyse de l'Expérience
    **Niveau** : [Junior / Confirmé / Senior]
    **Secteurs dominants** : [Indiquer les industries]
    **Atouts clés** : [Lister les points forts professionnels]

    ### 5. Formation Académique
    [Lister uniquement les diplômes et certificats obtenus]
    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

    # 4. Formatage
    text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    print("\n=== DÉBUT DE LA SYNTHÈSE (STREAMING) ===\n")

    # 5. Génération
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=1000,   # Une synthèse est plus courte qu'une réécriture complète
        temperature=0.2,       # Température basse pour être factuel
        top_p=0.9,
        repetition_penalty=1.1, # Évite de répéter les mêmes compétences
        streamer=streamer
    )

    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    full_response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

    return full_response

# ... (Tout le code précédent reste identique) ...

def save_to_txt(content, filename):
    """Sauvegarde le contenu dans un fichier texte."""
    try:
        with open(filename, "w", encoding="utf-8") as f:
            f.write(content)
        print(f"\n[OK] Résultat sauvegardé dans : {filename}")
    except Exception as e:
        print(f"\n[ERREUR] Impossible de sauvegarder le fichier : {e}")

# --- Test et Sauvegarde ---
if __name__ == "__main__":
    # 1. Définition des fichiers
    fichier_entree = "resultat_cv_celyann.txt"
    fichier_sortie = "synthese_finale_celyan.txt"  # Nom du fichier de sortie
    
    # 2. Lancement de l'analyse
    # Le texte s'affiche en direct grâce au Streamer
    resultat_final = synthesize_cv_for_matching(fichier_entree)
    
    # 3. Sauvegarde dans le fichier .txt
    save_to_txt(resultat_final, fichier_sortie)
    
    print("\n=== TRAITEMENT ET SAUVEGARDE TERMINÉS ===")


# calcul la similarité cosinus à partir d'embedding généré avec bge-m3

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

def match_cv_jobs():
    # 1. Chargement du modèle demandé
    print("Chargement du modèle 'BAAI/bge-m3'...")
    model = SentenceTransformer('BAAI/bge-m3')

    # 2. Lecture des fichiers
    try:
        # Lecture du CV
        with open('resultat_cv_celyann.txt', 'r', encoding='utf-8') as f:
            cv_text = f.read()
        
        # Lecture des annonces
        df_jobs = pd.read_csv('offres_resumees_qwen_live.csv')
        df_jobs = df_jobs.fillna('') # Remplacer les NaN par des chaînes vides
    except Exception as e:
        print(f"Erreur de lecture des fichiers : {e}")
        return

    # 3. Préparation des données (MODIFIÉ)
    # On combine uniquement : Poste + Entreprise + Resume_IA
    print("Préparation des données avec les colonnes : Poste, Entreprise, Resume_IA...")
    
    df_jobs['text_complet'] = (
        df_jobs['Poste'].astype(str) + " " + 
        df_jobs['Entreprise'].astype(str) + " " + 
        df_jobs['Resume_IA'].astype(str)
    )

    print(f"Vectorisation de {len(df_jobs)} offres en cours...")

    # 4. Vectorisation (Embeddings)
    # Le modèle transforme le texte en vecteurs numériques
    cv_vector = model.encode([cv_text])
    job_vectors = model.encode(df_jobs['text_complet'].tolist())

    # 5. Calcul de similarité
    scores = cosine_similarity(cv_vector, job_vectors)[0]
        
    # Ajout du score (en pourcentage)
    df_jobs['match_score'] = scores * 100
    
    # 6. Classement des résultats
    df_result = df_jobs.sort_values(by='match_score', ascending=False)
    
    # Affichage des meilleurs résultats
    # J'ai ajouté 'Resume_IA' à l'affichage pour que tu puisses vérifier la pertinence
    cols_to_show = ['Poste', 'Entreprise', 'match_score', 'Resume_IA', 'Lien']
    
    print("\n--- TOP 5 OFFRES CORRESPONDANTES ---")
    # On limite l'affichage de Resume_IA aux 100 premiers caractères pour la lisibilité console
    pd.set_option('display.max_colwidth', 100) 
    print(df_result[cols_to_show].head(5).to_string(index=False))
    
    # Sauvegarde
    output_filename = 'resultats_matching_resume_ia.csv'
    df_result.to_csv(output_filename, index=False)
    print(f"\nLes résultats complets ont été sauvegardés dans '{output_filename}'")

if __name__ == "__main__":
    match_cv_jobs()